In [7]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import json, requests, pandas as pd
import datetime

StatementMeta(, 6ec64016-94b0-4d7e-bd74-eb4a819cc97a, 9, Finished, Available)

In [8]:
def AuthenticateWithSP():
    key_vault = ""
    tenant = ""
    client = ""
    client_secret = mssparkutils.credentials.getSecret(key_vault , "clientsecret")   
 
    try:
        from azure.identity import ClientSecretCredential
    except Exception:
        !pip install azure.identity
        from azure.identity import ClientSecretCredential
 
    # Generates the access token for the Service Principal
    api = 'https://graph.microsoft.com/.default'
    auth = ClientSecretCredential(authority = 'https://login.microsoftonline.com/',
                              tenant_id = tenant,
                              client_id = client,
                              client_secret = client_secret)
    access_token = auth.get_token(api)
    access_token = access_token.token
    header = {'Authorization': f'Bearer {access_token}'}
    print('\nSuccessfully authenticated.')
    return header


StatementMeta(, 6ec64016-94b0-4d7e-bd74-eb4a819cc97a, 10, Finished, Available)

In [9]:
header= AuthenticateWithSP()

StatementMeta(, 6ec64016-94b0-4d7e-bd74-eb4a819cc97a, 11, Finished, Available)


Successfully authenticated.


In [10]:
def GetGroupMember(groupId, header,workspaceId, UserAccessRight):
    
    schema = StructType(
    [
        StructField('displayName', StringType(), True),
        StructField('id', StringType(), True),
        StructField('userPrincipalName', StringType(), True),
        StructField('userType', StringType(), True),
    ]
    )

    base_url = 'https://graph.microsoft.com/v1.0'
    API_url = base_url+"/groups/"+groupId+"/transitiveMembers?$select=displayName,id,userPrincipalName,userType"
    response = requests.get(API_url, headers=header)
    if response != None and response.status_code == 200:
        JSONContent = json.loads(response.content)
        JSONContentExplode = JSONContent['value']
        df_JSONContentExplode=spark.createDataFrame(data=JSONContentExplode, schema=schema)
        df_JSONContentExplode=df_JSONContentExplode\
                                                   .withColumn("workspaceId",lit(workspaceId) )\
                                                   .withColumn("accessRight",lit(UserAccessRight) )
        list_JSONContentExplode=df_JSONContentExplode.collect()
        return list_JSONContentExplode

StatementMeta(, 6ec64016-94b0-4d7e-bd74-eb4a819cc97a, 12, Finished, Available)

In [11]:
Workspace_users_groups= spark.read.format("delta").load("Files/Workspace_users_groups")

Workspace_users = Workspace_users_groups.select("WorkspaceId", "emailAddress","displayNAme","identifier","groupUserAccessRight","principalType")\
                                        .filter(((col("principalType")=="User") & (col("emailAddress").isNotNull())))\
                                        .withColumnRenamed("emailAddress", "UserPrincipalName")\
                                        .withColumnRenamed("groupUserAccessRight", "AccessRight")\
                                        .select("WorkspaceID","UserPrincipalName","AccessRight" )

Workspace_groups = Workspace_users_groups.select("WorkspaceId", "emailAddress","displayNAme","identifier","groupUserAccessRight","principalType")\
                                        .filter(((col("principalType")=="Group") & (col("emailAddress").isNotNull())))

StatementMeta(, 6ec64016-94b0-4d7e-bd74-eb4a819cc97a, 13, Finished, Available)

In [12]:
# get nested members on 3 levels

Workspace_groups_list=Workspace_groups.select("WorkspaceId","identifier","groupUserAccessRight").collect()
members= []
for i in Workspace_groups_list:
    result= GetGroupMember(i.identifier,header, i.WorkspaceId, i.groupUserAccessRight)
    if result != None:
        members.extend(result)

"""for i in members:
    if i.userType !="Member":
        result= GetGroupMember(i.id, header, i.WorkspaceID, i.AccessRight)
        members.remove(i)
        members.extend(result)

for i in members:
    if i.userType !="Member":
        result= GetGroupMember(i.id, header, i.WorkspaceID, i.AccessRight)
        members.remove(i)
        members.append(result)"""

Workspace_usersFromGroups= spark.createDataFrame(data=members)
Workspace_usersFromGroups= Workspace_usersFromGroups.withColumnRenamed("workspaceId", "WorkspaceID")\
                                                    .withColumnRenamed("userPrincipalName", "UserPrincipalName")\
                                                    .withColumnRenamed("accessRight", "AccessRight")\
                                                    .select("WorkspaceID","UserPrincipalName","AccessRight")

StatementMeta(, 6ec64016-94b0-4d7e-bd74-eb4a819cc97a, 14, Finished, Available)

In [13]:
Workspace_users_result = Workspace_users.unionByName(Workspace_usersFromGroups).dropDuplicates()

StatementMeta(, 6ec64016-94b0-4d7e-bd74-eb4a819cc97a, 15, Finished, Available)

In [14]:
# add the company name 
Users_AD= spark.read.format("delta").load("Files/AD_users")

Workspace_users_result=Workspace_users_result.join(Users_AD,Workspace_users_result.UserPrincipalName== Users_AD.userPrincipalName,"left")\
                        .select("WorkspaceID",Workspace_users_result.UserPrincipalName,"AccessRight","companyName")\
                        .withColumnRenamed("companyName","CompanyName")

StatementMeta(, 6ec64016-94b0-4d7e-bd74-eb4a819cc97a, 16, Finished, Available)

In [15]:
Workspace_users_result.write.format('delta')\
.mode("overwrite")\
.option("overwriteSchema", "True")\
.save("Files/Workspace_users")

StatementMeta(, 6ec64016-94b0-4d7e-bd74-eb4a819cc97a, 17, Finished, Available)